In [1]:
# Vol Surface Constructor

# 1/ Collect Data from YF
# 2/ Process and clean data
# 3/ Calculate IV
# 4/ Contsruct Vol Surface

In [2]:
# Updates

# Make the code more pythonic (use functions and call them rather than jsut blocks of code)
      # ---> made progress with this (almsot done)

# maybe add logic where we can mnaually remove outliers


# still dont fully understand RBF

# incorporate PDE into this model (algebraic continuation YT video on quant roadmap 2025)

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from scipy.interpolate import Rbf
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go

In [4]:
# INPUTS
ticker = 'SPY'
outlier_penalty = 1
k = 5   # no. of neighbours in KNN
smooth_input = 0.5  # used for RBF Interpolation
minIV = 0.0001

In [5]:
# Import data and convert into dataframe
def import_data(ticker):
    stock_data = yf.Ticker(ticker)
    expiration_dates = stock_data.options
    option_data = []

    # Loop through all expiration dates and collect the data
    for exp in expiration_dates:
        option_chain = stock_data.option_chain(exp)
        
        calls = option_chain.calls
        puts = option_chain.puts
        
        calls['expiration_date'] = exp
        calls['type'] = 'call'
        puts['expiration_date'] = exp
        puts['type'] = 'put'
        
        option_data.append(calls)
        option_data.append(puts)

    options_df = pd.DataFrame(pd.concat(option_data, ignore_index=True))

    return options_df

options_df = import_data(ticker)


In [6]:
# Data Cleaning

def clean_data(options_df):
    # Convert expiration date to datetime
    options_df["expiration_date"] = pd.to_datetime(options_df["expiration_date"])

    # remove rows with 'null' values
    options_df = options_df[~options_df.isnull().any(axis=1)]

    # drop duplicates
    options_df = options_df.drop_duplicates(subset="contractSymbol")

    # drop columns
    options_df.drop(columns=["lastTradeDate","change","percentChange","volume","inTheMoney","contractSize","currency"],inplace=True, errors="ignore")

    # Normalise strike price
    current_price = yf.Ticker(ticker).history(period="1d").Close.iloc[-1]
    options_df["moneyness"] = options_df["strike"] / current_price
    options_df = options_df[(options_df["moneyness"] <= 1.2) & (options_df["moneyness"] >= 0.8)]

    # time to maturity (+ filtering out long date options)
    options_df["time_to_maturity"] = (options_df.expiration_date - pd.to_datetime("today")).dt.days
    options_df = options_df[(options_df["time_to_maturity"] <= 180) & (options_df["time_to_maturity"] > 0)]

    # Filter out thinly traded options (low open interest)
    options_df = options_df[options_df.openInterest > 1000] # may need to change this -> this would need to change for diff tickers

    # Filter out extremely low IVs
    options_df = options_df[options_df["impliedVolatility"] > minIV]

    return options_df

options_df = clean_data(options_df)



In [7]:
# filter calls and puts

def option_type_filter(options_df, option_type):
    return options_df[options_df['type'] == option_type]


calls_df = option_type_filter(options_df, 'call')
puts_df = option_type_filter(options_df, 'put')

In [8]:
# Outlier Detection 

# step 1 -> standaridse moneyness and time to maturity as KNN is sensitive to scale

def outlier_detection(df, outlier_penalty):

    # Initialise the MinMaxScaler
    scaler = StandardScaler()

    # Normalise both features
    scaled_data = scaler.fit_transform(df[['moneyness', 'time_to_maturity']])

    IV = df['impliedVolatility'].values

    # Fit a KNN model to the feature data
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(scaled_data)

    # Find the k nearest neighbors for each point
    distances, indices = knn.kneighbors(scaled_data)

    # Calculate the average IV of the neighbors
    avg_IV_neighbors = []
    for i in range(len(df)):
        # Get the IVs of the k nearest neighbors
        neighbor_IVs = IV[indices[i]]
        avg_IV_neighbors.append(np.mean(neighbor_IVs))

    # Compute the difference between the IV of each point and its neighbors' average IV
    iv_diff = np.abs(IV - avg_IV_neighbors)

    # Define a threshold for outliers 
    threshold = outlier_penalty * np.std(iv_diff)

    # Flag outliers
    outliers = iv_diff > threshold

    # Add the 'Outlier' column to the DataFrame
    df['Outlier'] = outliers

    # df["Outlier_Adj_IV"] = np.where(df["Outlier"] == True, np.nan, df["impliedVolatility"])

    return df


outlier_detection(calls_df, outlier_penalty)
outlier_detection(puts_df, outlier_penalty)

C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_40916\2360725369.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = outliers
C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_40916\2360725369.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = outliers


,contractSymbol,strike,lastPrice,bid,ask,openInterest,impliedVolatility,expiration_date,type,moneyness,time_to_maturity,Outlier
41,SPY250113P00550000,550.0,0.01,0.01,0.02,6482.0,0.232430,2025-01-13,put,0.947475,1,True
44,SPY250113P00565000,565.0,0.05,0.04,0.05,1457.0,0.143075,2025-01-13,put,0.973316,1,True
45,SPY250113P00570000,570.0,0.13,0.13,0.14,3463.0,0.123544,2025-01-13,put,0.981929,1,True
46,SPY250113P00575000,575.0,0.60,0.60,0.61,5410.0,0.116220,2025-01-13,put,0.990542,1,True
47,SPY250113P00580000,580.0,2.10,2.06,2.09,4541.0,0.110849,2025-01-13,put,0.999156,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4231,SPY250630P00570000,570.0,17.59,18.29,18.37,1194.0,0.148095,2025-06-30,put,0.981929,169,False
4233,SPY250630P00580000,580.0,21.60,21.32,21.41,2340.0,0.136804,2025-06-30,put,0.999156,169,False
4234,SPY250630P00585000,585.0,23.11,23.08,23.18,1717.0,0.131158,2025-06-30,put,1.007769,169,False
4235,SPY250630P00590000,590.0,25.17,25.00,25.13,1339.0,0.125398,2025-06-30,put,1.016383,169,False


In [9]:
# create equivalent dataframes that exclude Outliers, this will be useful for interpolation
calls_df_no_nan = calls_df[calls_df.Outlier == False]
puts_df_no_nan = puts_df[puts_df.Outlier == False]



In [10]:
# RBFs

# It assigns each data point a weight that diminishes the further it is from the target (modelled by an RBF)
# Interpolated value at target point is then a weighted sum of the known values

In [11]:
# this function requries 
    # df with outliers
    # df without outliers

# Robust Scaler is used as it deals with outliers better (it works off IQR)

def construct_vol_surface(df, df_no_nan):

    scaler = RobustScaler()

    X_no_nan = df_no_nan[['moneyness', 'time_to_maturity']].values

    X_scaled_no_nan = scaler.fit_transform(X_no_nan)

    strike_grid_scaled = np.linspace(X_scaled_no_nan[:, 0].min(), X_scaled_no_nan[:, 0].max(), 50)
    maturity_grid_scaled = np.linspace(X_scaled_no_nan[:, 1].min(), X_scaled_no_nan[:, 1].max(), 50)

    X_scaled_grid, Y_scaled_grid = np.meshgrid(strike_grid_scaled, maturity_grid_scaled)

    rbf = Rbf(X_scaled_no_nan[:, 0], X_scaled_no_nan[:, 1], df_no_nan['impliedVolatility'], function='linear', smooth=smooth_input)
    Z_scaled = rbf(X_scaled_grid, Y_scaled_grid)

    X_rescaled = scaler.inverse_transform(np.column_stack([X_scaled_grid.ravel(), Y_scaled_grid.ravel()]))  # Inverse scaling
    X_rescaled = X_rescaled.reshape(X_scaled_grid.shape + (2,))  # Shape back into meshgrid dimensions


    # Surface plot for the RBF interpolated implied volatility
    surface = go.Surface(
        x=X_rescaled[:, :, 0],  # Rescaled moneyness
        y=X_rescaled[:, :, 1],  # Rescaled time to maturity
        z=Z_scaled.reshape(X_scaled_grid.shape),  # Interpolated volatility
        opacity=0.7,
        name='Smoothed Surface',
        colorscale="Jet"
    )

    # Define the scatter plot, using the 'Outlier_Int' for coloring
    scatter = go.Scatter3d(
        x=df['moneyness'],
        y=df['time_to_maturity'],
        z=df['impliedVolatility'],  # Using the original IV (includes outliers)
        mode='markers',
        marker=dict(
            size=2,
            color="black"),
        text=df["Outlier"],
        hovertemplate=(
            'Moneyness: %{x}<br>'       
            'Time to Maturity: %{y}<br>'
            'Implied Volatility: %{z}<br>'
            'Outlier: %{text}<br>'))

    fig = go.Figure(data=[scatter, surface])
    fig.update_layout(
            title="Volatility Surface - {ticker}",
            scene=dict(
            xaxis_title='Moneyness',
            yaxis_title='Time to Maturity',
            zaxis_title='Implied Volatility'))

    fig.show()




construct_vol_surface(calls_df, calls_df_no_nan)
construct_vol_surface(puts_df, puts_df_no_nan)

In [18]:
scaler = RobustScaler()

X_no_nan = calls_df_no_nan[['moneyness', 'time_to_maturity']].values

X_scaled_no_nan = scaler.fit_transform(X_no_nan)

strike_grid_scaled = np.linspace(X_scaled_no_nan[:, 0].min(), X_scaled_no_nan[:, 0].max(), 50)
maturity_grid_scaled = np.linspace(X_scaled_no_nan[:, 1].min(), X_scaled_no_nan[:, 1].max(), 50)

X_scaled_grid, Y_scaled_grid = np.meshgrid(strike_grid_scaled, maturity_grid_scaled)

rbf = Rbf(X_scaled_no_nan[:, 0], X_scaled_no_nan[:, 1], calls_df_no_nan['impliedVolatility'], function='linear', smooth=smooth_input)
Z_scaled = rbf(X_scaled_grid, Y_scaled_grid)

X_rescaled = scaler.inverse_transform(np.column_stack([X_scaled_grid.ravel(), Y_scaled_grid.ravel()]))  # Inverse scaling
X_rescaled = X_rescaled.reshape(X_scaled_grid.shape + (2,))  # Shape back into meshgrid dimensions


z=Z_scaled.reshape(X_scaled_grid.shape),  # Interpolated volatility


z

(array([[0.67568829, 0.66253482, 0.64349703, ..., 0.36481821, 0.3632201 ,
         0.36269741],
        [0.67103101, 0.6568374 , 0.63685805, ..., 0.35631174, 0.35345677,
         0.35317075],
        [0.66084262, 0.64703453, 0.62725719, ..., 0.34271492, 0.34200136,
         0.34272351],
        ...,
        [0.38752713, 0.37071703, 0.3575268 , ..., 0.13139626, 0.13571388,
         0.14290222],
        [0.38853484, 0.37244099, 0.35916379, ..., 0.13353484, 0.13839659,
         0.14590357],
        [0.38990927, 0.37435584, 0.36111225, ..., 0.13612207, 0.14143958,
         0.14910396]], shape=(50, 50)),)

In [19]:
import plotly.graph_objects as go
import numpy as np

def plot_2d_graph_interactive(df, df_no_nan, rbf):
    # Dropdown and input setup for interactive controls
    from ipywidgets import widgets
    from IPython.display import display
    
    # Define options for dropdown (X-axis choice)
    x_axis_dropdown = widgets.Dropdown(
        options=['moneyness', 'time_to_maturity'],
        value='moneyness',  # Default
        description='X-axis:'
    )

    # Slider for moneyness selection when time_to_maturity is on the X-axis
    moneyness_slider = widgets.FloatSlider(
        value=1.0,  # Start with ATM (moneyness = 1.0)
        min=df['moneyness'].min(),
        max=df['moneyness'].max(),
        step=0.01,
        description='Moneyness:'
    )

    # Function to update the plot based on user selection
    def update_plot(x_axis_choice, moneyness_value):
        # Choose the data to use for the x-axis
        if x_axis_choice == 'moneyness':
            x_data = df['moneyness']
            x_range = np.linspace(df_no_nan['moneyness'].min(), df_no_nan['moneyness'].max(), 100)
            y_data = df['time_to_maturity']  # Fixed time to maturity for each moneyness
            # Interpolated volatility for the range of moneyness
            interpolated_vol = rbf(x_range, np.full_like(x_range, moneyness_value))
        else:  # time_to_maturity
            x_data = df['time_to_maturity']
            x_range = np.linspace(df_no_nan['time_to_maturity'].min(), df_no_nan['time_to_maturity'].max(), 100)
            y_data = np.full_like(x_range, moneyness_value)  # Fixed moneyness value
            # Interpolated volatility for the range of time to maturities
            interpolated_vol = rbf(np.full_like(x_range, moneyness_value), x_range)

        # Create scatter plot for actual data points
        scatter = go.Scatter(
            x=x_data,
            y=df['impliedVolatility'],
            mode='markers',
            marker=dict(size=6, color='black'),
            name='Actual Data Points'
        )

        # Create the line plot for the interpolated volatility surface
        surface_line = go.Scatter(
            x=x_range,
            y=interpolated_vol,
            mode='lines',
            line=dict(color='blue'),
            name='Interpolated Volatility Surface'
        )

        # Combine the plots in one figure
        fig = go.Figure(data=[scatter, surface_line])
        fig.update_layout(
            title=f'Implied Volatility vs. {x_axis_choice.capitalize()}',
            xaxis_title=f'{x_axis_choice.capitalize()}',
            yaxis_title='Implied Volatility',
            showlegend=True
        )

        fig.show()

    # Display interactive elements and plot
    widgets.interactive(update_plot, x_axis_choice=x_axis_dropdown, moneyness_value=moneyness_slider)

# Usage (Make sure 'rbf' is the RBF model you've already constructed)
plot_2d_graph_interactive(calls_df, calls_df_no_nan, rbf)  # This will show interactive plot for calls data


    

